In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn import svm
import pandas as pd
import glob
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
from ast import literal_eval
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, confusion_matrix, ConfusionMatrixDisplay
from sklearn import svm
from sklearn.model_selection import RandomizedSearchCV, train_test_split
import numpy as np
from sklearn.preprocessing import StandardScaler

In [ ]:
def data_frame_adder(temp_frame,file_name):
    file_name=file_name.split('.csv')[0]
    temp_frame['file_name']=file_name
    temp_frame['subject']=temp_frame['file_name'].apply(lambda x: int(x.split('subject_')[1].split('_')[0]))
    temp_frame['session']=temp_frame['file_name'].apply(lambda x: int(x.split('subject_')[1].split('_')[1]))
    #temp_frame['type']=temp_frame['file_name'].apply(lambda x: x.split('__')[1])
    temp_frame = temp_frame.drop(columns=['file_name'])
    temp_frame = temp_frame[temp_frame.columns[::-1]]
    temp_frame = temp_frame.sort_values(by=['session','subject'], axis=0)
    col_list1 = list(temp_frame.columns)[0:2]
    col_list2 = list(temp_frame.columns)[2:]
    col_list2.sort()
    col_list=col_list1+col_list2
    
    #temp_frame = swap_columns(temp_frame, 'subject', 'session')
    # Swapping the columns
    temp_frame = temp_frame.reindex(columns=col_list)

    return temp_frame

In [3]:

Training_data_files=glob.glob('TrainingData/*.csv')
file_name_list_x=[]
file_name_list_x_time=[]
file_name_list_y=[]
file_name_list_y_time=[]
for file_name in tqdm(Training_data_files):
    if 'x.csv' in file_name:
        file_name_list_x.append(file_name)
    elif 'x_time.csv' in file_name:
        file_name_list_x_time.append(file_name)
    elif 'y.csv' in file_name:
        file_name_list_y.append(file_name)
    else:
        file_name_list_y_time.append(file_name)
file_name_list_x.sort()
file_name_list_x_time.sort()
file_name_list_y.sort()
file_name_list_y_time.sort()
Subject_list= [f.split('_')[1] for f in file_name_list_x]
Session_list= [f.split('_')[2] for f in file_name_list_x]

x_sensor_dat=pd.DataFrame([])
y_sensor_dat=pd.DataFrame([])
x_time_dat=[]
for sub,sess in tqdm(zip(Subject_list,Session_list)):
    x_sensor_dat=pd.concat([pd.read_csv(f"TrainingData/subject_{sub}_{sess}__x.csv",header=None),pd.read_csv(f"TrainingData/subject_{sub}_{sess}__x_time.csv",header=None).rename(columns={0:'time_stamp'})],axis=1)
    features_to_normalize = [0,1,2,3,4,5]
    features_to_normalize = [0,1,2,3,4,5]
    scaler = StandardScaler()
    scaler.fit(x_sensor_dat[features_to_normalize])
    scaled = scaler.fit_transform(x_sensor_dat[features_to_normalize])
    scaled_df = pd.DataFrame(scaled, columns=x_sensor_dat[features_to_normalize].columns)
    x_sensor_dat[features_to_normalize]=scaled_df
    break

100%|███████████████████████████████████████████████| 116/116 [00:00<00:00, 967274.88it/s]
0it [00:00, ?it/s]


In [9]:

Training_data_files=glob.glob('TrainingData/*.csv')
file_name_list_x=[]
file_name_list_x_time=[]
file_name_list_y=[]
file_name_list_y_time=[]
for file_name in tqdm(Training_data_files):
    if 'x.csv' in file_name:
        file_name_list_x.append(file_name)
    elif 'x_time.csv' in file_name:
        file_name_list_x_time.append(file_name)
    elif 'y.csv' in file_name:
        file_name_list_y.append(file_name)
    else:
        file_name_list_y_time.append(file_name)
file_name_list_x.sort()
file_name_list_x_time.sort()
file_name_list_y.sort()
file_name_list_y_time.sort()
Subject_list= [f.split('_')[1] for f in file_name_list_x]
Session_list= [f.split('_')[2] for f in file_name_list_x]

x_sensor_dat=pd.DataFrame([])
y_sensor_dat=pd.DataFrame([])
x_time_dat=[]
for sub,sess in tqdm(zip(Subject_list,Session_list)):
    x_sensor_dat=pd.concat([pd.read_csv(f"TrainingData/subject_{sub}_{sess}__x.csv",header=None),pd.read_csv(f"TrainingData/subject_{sub}_{sess}__x_time.csv",header=None).rename(columns={0:'time_stamp'})],axis=1)
    features_to_normalize = [0,1,2,3,4,5]
    features_to_normalize = [0,1,2,3,4,5]
    scaler = StandardScaler()
    scaler.fit(x_sensor_dat[features_to_normalize])
    scaled = scaler.fit_transform(x_sensor_dat[features_to_normalize])
    scaled_df = pd.DataFrame(scaled, columns=x_sensor_dat[features_to_normalize].columns)
    x_sensor_dat[features_to_normalize]=scaled_df
    x_sensor_dat['measurements']=x_sensor_dat.apply(lambda x: [x[0],x[1],x[2],x[3],x[4],x[5]],axis=1)
    x_sensor_dat=x_sensor_dat[['time_stamp','measurements']]
    x_sensor_dat['Subject']=int(sub)
    x_sensor_dat['Session']=int(sess)
    
    y_sensor_dat=pd.concat([pd.read_csv(f"TrainingData/subject_{sub}_{sess}__y.csv",header=None),pd.read_csv(f"TrainingData/subject_{sub}_{sess}__y_time.csv",header=None).rename(columns={0:'time_stamp'})],axis=1).rename(columns={0:'labels'})
    y_sensor_dat=y_sensor_dat[['time_stamp','labels']]
    trp=pd.concat(
        [pd.DataFrame({'labels':0},index=[x_sensor_dat.index[-1]]),pd.DataFrame({'time_stamp':np.max(x_sensor_dat['time_stamp'])},index=[x_sensor_dat.index[-1]])
                         ]
         ,axis=1)
    break
    trp
    y_sensor_dat=pd.concat([y_sensor_dat,trp],axis=0)
    trp=pd.concat([pd.DataFrame({'labels':0},index=[x_sensor_dat.index[0]]),pd.DataFrame({'time_stamp':np.min(x_sensor_dat['time_stamp'])},index=[x_sensor_dat.index[0]])],axis=1)
    y_sensor_dat=pd.concat([trp,y_sensor_dat],axis=0)
    # y_sensor_dat=y_sensor_dat.set_index('time_stamp').asfreq('0.025S')
    #y_sensor_dat['Subject']=int(sub)
    #y_sensor_dat['Session']=int(sess)
    x_sensor_dat["time_stamp"]=pd.to_datetime(x_sensor_dat["time_stamp"],unit='s').round('ms')
    x_sensor_dat=x_sensor_dat.set_index('time_stamp')
    x_sensor_dat=x_sensor_dat.asfreq('0.025S')
    
    y_sensor_dat["time_stamp"]=pd.to_datetime(y_sensor_dat["time_stamp"],unit='s').round('ms')
    y_sensor_dat=y_sensor_dat.set_index('time_stamp')
    y_sensor_dat=y_sensor_dat.asfreq(freq='0.025S', method='bfill')
    x_y_dat=pd.merge(x_sensor_dat,y_sensor_dat, how='inner', left_index=True, right_index=True)
    x_y_dat.to_parquet(f"pre_processed_training/subject_{sub}_session_{sess}.gzip",compression='gzip')


100%|███████████████████████████████████████████████| 116/116 [00:00<00:00, 921475.88it/s]
0it [00:00, ?it/s]


In [10]:
y_sensor_dat

,time_stamp,labels
0,0.02,0
1,0.12,0
2,0.22,0
3,0.32,0
4,0.42,0
...,...,...
9468,946.82,0
9469,946.92,0
9470,947.02,0
9471,947.12,0


In [11]:
x_sensor_dat

,time_stamp,measurements,Subject,Session
0,0.000,"[0.49225123220935507, 0.0574821663159993, -0.5...",1,1
1,0.025,"[0.45489722483915207, 0.07778523441849579, -0....",1,1
2,0.050,"[0.5086999002612168, 0.08656965800233246, -0.5...",1,1
3,0.075,"[0.5545241549751831, 0.08697396258912203, -0.5...",1,1
4,0.100,"[0.5033684694494693, 0.046890343885634525, -0....",1,1
...,...,...,...,...
37885,947.125,"[0.11504486918898627, 0.24747590569891953, -0....",1,1
37886,947.150,"[0.11544359469698251, 0.24446038180963306, -0....",1,1
37887,947.175,"[0.11185551634211324, 0.24346200259819348, -0....",1,1
37888,947.200,"[0.11098857789319193, 0.24617501635502942, -0....",1,1


In [ ]:
files=glob.glob('pre_processed_training/*.gzip')
total_training_data=pd.DataFrame([])
for f in tqdm(files):
    total_training_data=pd.concat([total_training_data,pd.read_parquet(f, engine='auto')],axis=0)
total_training_data=total_training_data.sort_values(['Subject','Session','time_stamp'])
total_training_data=total_training_data.reset_index()
total_training_data['epoch'] = total_training_data['time_stamp'].sub(pd.Timestamp('1970-01-01 00:00:00.000'))
total_training_data['epoch']=total_training_data['epoch'].dt.total_seconds()
total_training_data=total_training_data[['Subject','Session','epoch','measurements','labels']]
total_training_data_grouped=total_training_data.groupby(['Subject','Session','epoch']).agg({'measurements':[np.mean],'labels':[np.mean]})
total_training_data_grouped.columns = total_training_data_grouped.columns.droplevel(1)
total_training_data_grouped=total_training_data_grouped.reset_index(level=['epoch'])
total_training_data_grouped.to_parquet(f"combined_sampled/training.gzip",compression='gzip')